In [ ]:
import os
import random
import numpy as np
import tensorflow as tf


seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.config.experimental.enable_op_determinism()
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

print(f"Reproductibilité activée: seed={seed}")

In [ ]:
from tensorflow import keras
from keras import layers, models
import pyarrow as pa
import pyarrow.parquet as pq
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers,models,callbacks
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score,calinski_harabasz_score, silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
from umap import UMAP
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
import time
from io import StringIO
from joblib import Parallel, delayed
import pickle
import networkx as nx



os.chdir(os.path.expanduser("~/projets/pro/BIP12: Artificial Intelligence"))    #Change as needed
print(f"Working Directory:\n{os.getcwd()}\n")
print(f"Keras version= {keras.__version__}")
print(f"Pandas version= {pd.__version__}")

#Ensure reproductibility
tf.config.experimental.enable_op_determinism()


In [ ]:
df = pd.read_excel("data/mmc2.xlsx", header=1)
df = df.dropna(axis=0, how='all')
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)
df = df.rename(columns={"Case.ID": "Case_ID"})
df.head()


## Téléchargement des données RNAseq à partir des CASE ID de chaque patient dans la liste cohort

In [ ]:
def download_tcga_rnaseq(case_id):
    """Télécharge les données RNAseq pour un case_id TCGA"""
    url = "https://api.gdc.cancer.gov/files"
    
    filters = {
        "op": "and",
        "content": [
            {"op": "in", "content": {"field": "cases.submitter_id", "value": [case_id]}},
            {"op": "in", "content": {"field": "files.data_type", "value": ["Gene Expression Quantification"]}},
            {"op": "in", "content": {"field": "files.analysis.workflow_type", "value": ["STAR - Counts"]}}
        ]
    }
    
    params = {
        "filters": json.dumps(filters),
        "fields": "file_id,file_name,cases.submitter_id",
        "format": "JSON",
        "size": "100"
    }
    
    response = requests.get(url, params=params)
    return response.json()

# Tester avec un patient
test_case = df["Case_ID"].iloc[0]
result = download_tcga_rnaseq(test_case)
print(f"Fichiers trouvés pour {test_case}: {result['data']['pagination']['total']}")
print(result)

In [ ]:
# 1. Fonction pour télécharger les données d'un fichier
def download_gene_counts(file_id):
    """Télécharge le fichier de counts depuis GDC"""
    url = f"https://api.gdc.cancer.gov/data/{file_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Lire le TSV
        data = pd.read_csv(StringIO(response.text), sep='\t', comment='#')
        return data
    else:
        return None

# 2. Construire la matrice pour tous les patients
def build_expression_matrix(case_ids, max_patients=None):
    """
    Construit la matrice d'expression pour une liste de patients
    
    Parameters:
    -----------
    case_ids : list
        Liste des Case_ID TCGA
    max_patients : int, optional
        Limite le nombre de patients (pour tester)
    
    Returns:
    --------
    pd.DataFrame : Matrice patients × gènes
    """
    
    expression_data = {}
    failed_cases = []
    
    # Limiter si spécifié
    if max_patients:
        case_ids = case_ids[:max_patients]
    
    print(f"Téléchargement des données pour {len(case_ids)} patients...")
    
    for i, case_id in enumerate(case_ids):
        try:
            # Récupérer le file_id
            result = download_tcga_rnaseq(case_id)
            
            if result['data']['pagination']['total'] > 0:
                file_id = result['data']['hits'][0]['file_id']
                
                # Télécharger les counts
                counts = download_gene_counts(file_id)
                
                if counts is not None:
                    # Extraire les counts (colonne 'unstranded' ou 'tpm_unstranded')
                    # Adapter selon le format de vos fichiers
                    gene_counts = counts.set_index('gene_name')['unstranded']
                    expression_data[case_id] = gene_counts
                    
                    if (i + 1) % 10 == 0:
                        print(f"Téléchargé: {i + 1}/{len(case_ids)}")
                else:
                    failed_cases.append(case_id)
            else:
                print(f"Pas de fichier pour {case_id}")
                failed_cases.append(case_id)
            
            # Respecter les limites de l'API
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Erreur pour {case_id}: {e}")
            failed_cases.append(case_id)
    
    # Créer le DataFrame (patients en lignes, gènes en colonnes)
    expression_matrix = pd.DataFrame(expression_data).T
    
    print(f"\nMatrice construite: {expression_matrix.shape}")
    print(f"  - Patients: {expression_matrix.shape[0]}")
    print(f"  - Gènes: {expression_matrix.shape[1]}")
    if failed_cases:
        print(f"  - Échecs: {len(failed_cases)} patients")
    
    return expression_matrix, failed_cases

"""-------------------------------------------------------------"""

# 3. TESTER avec quelques patients d'abord
patients_with_rna = df[df['mRNA'].notna()]['Case_ID'].tolist()
print(f"Total patients avec mRNA: {len(patients_with_rna)}")

# TEST avec 5 patients
test_matrix, failed = build_expression_matrix(patients_with_rna, max_patients=817)


### Saving the resulting dataframe for future use without redownloading each time..then loading again


In [ ]:
nan_cols = [c for c in test_matrix.columns if pd.isna(c)]
print(f"{len(nan_cols)} colonnes NaN supprimées")
test_matrix = test_matrix.drop(columns=nan_cols)
test_matrix.to_pickle("dataframe.pkl")
df.to_pickle("data_cohort.pkl")
#test_matrix.head()
#data_cohort.head()
print("\nDonnées sauvegardées")

### Loading the pickled results

In [ ]:
test_matrix=pd.read_pickle("dataframe.pkl")
data_cohort=pd.read_pickle("data_cohort.pkl")
print(f"Données chargées: {test_matrix.shape}")
print(f"Méta données chargées: {data_cohort.shape}")

In [ ]:
expr = test_matrix.copy()
lib_size = expr.sum(axis=1)
cpm = (expr.T / lib_size).T * 1e6

#Filters
min_cpm = 1.0          # keep genes with CPM >= 1
min_prop = 0.20        # ...in at least 20% of samples
min_samples = int(np.ceil(min_prop * cpm.shape[0]))
keep = (cpm >= min_cpm).sum(axis=0) >= min_samples
cpm_filt = cpm.loc[:, keep]

print(f"Gènes gardés après le filtre d'expression: {cpm_filt.shape[1]} gènes")
log_cpm = np.log1p(cpm_filt)


# 4) HVG by variance (top-K)
top_k = 5000  
gene_var = log_cpm.var(axis=0)
hvg_genes = gene_var.sort_values(ascending=False).head(min(top_k, log_cpm.shape[1])).index

expr_hvg = log_cpm[hvg_genes].copy()
print(f"Gènes gardés après le filtre de variance HVG: {expr_hvg.shape[1]} gènes")
print(f"HVG matrix résultante: {expr_hvg.shape}")

# 5) Use HVGs downstream
test_matrix_hvg = expr_hvg

In [ ]:
# Vérifier d'abord les vrais noms des colonnes dans df
print(df.columns)
metadata_cols = ["TumorPurity", "Final Pathology", "ProliferationScore",'PAM50']

# Créer une dataframe avec les métadonnées, indexée par Case_ID
metadata = df[df['Case_ID'].isin(test_matrix.index)][['Case_ID'] + metadata_cols].set_index('Case_ID')

# Ajouter les métadonnées à test_matrix
test_matrix_with_meta = test_matrix_hvg.copy()
test_matrix_with_meta = test_matrix_with_meta.join(metadata, how='left')

print(f"\nColonnes des méta données à ajouter:\n{metadata_cols}")
print(f"\nShape avant l'ajout des méta données: {test_matrix_hvg.shape}")
print(f"Shape après l'ajout des méta données: {test_matrix_with_meta.shape}\n")
#print(test_matrix_with_meta.iloc[:, -5:].head())
#print(test_matrix_with_meta.head())

In [ ]:
test_matrix_clean=test_matrix_hvg.copy()
test_matrix_clean = test_matrix_clean.rename(columns={"gene_name": "Case_ID"})
test_matrix_clean.shape
#print(test_matrix_clean.iloc[:,-3:].head())
test_matrix_clean.shape

In [ ]:
pca=PCA(whiten=True,random_state=seed)
X_pca=pca.fit_transform(test_matrix_clean)
print("\n Data après PCA")
print(X_pca.shape)
cumsum_var = np.cumsum(pca.explained_variance_ratio_)

# Trouver le nombre de composants pour 85%
n_components_85 = np.argmax(cumsum_var >= 0.85) + 1

print(f"Nombre de PCs pour 85% variance: {n_components_85}")
print(f"Variance expliquée: {cumsum_var[n_components_85-1]:.4f}")

# Visualiser
plt.figure(figsize=(10, 5))
plt.plot(cumsum_var, linewidth=2, color="teal")
plt.axhline(y=0.85, color='r', linestyle='--', label='85%')
plt.axvline(x=n_components_85, color='b', linestyle='--', label=f'n={n_components_85}')
plt.title("Variance en fonction du nombre de PCs")
plt.xlabel("Nombre de PCs")
plt.ylabel("Variance cumulée")
plt.legend()
plt.grid()
plt.show()

In [ ]:
print(n_components_85)

pca_reduced=PCA(n_components=n_components_85,random_state=seed)
data_pca_reduced = pca_reduced.fit_transform(data_scaled)


print(f"Après PCA: {data_pca_reduced.shape}")